In case we don't have Bio libary


In [1]:
# !pip install Bio

Essential libaries

In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import GroupShuffleSplit
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold, KFold
import itertools
import re
from collections import Counter
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# from captum.attr import IntegratedGradients
# from captum.attr import LayerConductance
# from captum.attr import NeuronConductance
from Bio import SeqIO, SeqUtils

Calculation of DNA Codon usage, kmer usage, orf and general DNA features for usage

In [3]:
# codon usage features extraction
codons = [''.join(i) for i in list(itertools.product(['A', 'G', 'T', 'C'], repeat=3) )]


def calculate_codon_usage(seq):
    vector = []
    length = len(seq)
    a = Counter([str(seq[3*i:3*i+3]) for i in range(0, len(seq)//3)])
    b = {i:a[i] / length for i in a}
    for cod in codons:
        if cod in b:
            vector.append(b[cod])
        else:
            vector.append(0)
    return vector

def calculate_kmer_usage(seq, kmers_length):
    KMERS = [''.join(i) for i in list(itertools.product(['A', 'T', 'G', 'C'], repeat=kmers_length) )]
    vector = []
    length = len(seq)
    a = Counter([str(seq[i:i+kmers_length]) for i in range(0, len(seq) - kmers_length)])
    b = {i:a[i] / length for i in a}
    for kmer in KMERS:
        if kmer in b:
            vector.append(b[kmer])
        else:
            vector.append(0)
    return vector

# total ORF length feature extraction
def get_orfs_len(seq):
    orfs = [i[0] for i in re.findall("((ATG|GTG|TTG).{100,}?(TAG|TAA|TGA))", seq) if len(i[0]) % 3 == 0]
    return sum([len(i) for i in orfs]) / len(seq)

def extract_data_features(fasta_file):
    fasta = list(SeqIO.parse(fasta_file,  "fasta"))
    df = []
    for f in fasta:
        df.append((f.id, ''.join(f.seq)))
    df = pd.DataFrame(df, columns=['id', 'seq'])
    # extract_features
    df['len'] = df.seq.str.len()
    df['classification'] = df.id.str.split('-').str[0].str.replace('Bacteria', '0').str.replace('Phage', '1').astype(int)
    df['gc_content'] = df.seq.apply(SeqUtils.GC)
    df['ATG_freq'] = df.seq.str.count('ATG') / df.len # redundant with codon usage
    df['GTG_freqs'] = df.seq.str.count('GTG') / df.len # redundant with codon usage
    df['TTG_freq'] = df.seq.str.count('TTG') / df.len # redundant with codon usage
    df['A_freq'] = df.seq.str.count('A') / df.len
    df['G_freq'] = df.seq.str.count('G') / df.len
    df['C_freq'] = df.seq.str.count('C') / df.len
    df['T_freq'] = df.seq.str.count('T') / df.len # redundant
    df['extra_orfs_length'] = df.seq.apply(get_orfs_len)
    
    df['codon_usage_vec'] = df.seq.apply(calculate_codon_usage)
    df = pd.merge(df, df.codon_usage_vec.apply(pd.Series), right_index=True, left_index=True)
    df = df.drop(columns=['codon_usage_vec'])
    
    for i in [2, 3, 4]:
        df['kmer_usage_vec'] = df.seq.apply(lambda x: calculate_kmer_usage(x,i))
        df = pd.merge(df, df.kmer_usage_vec.apply(pd.Series), right_index=True, left_index=True)
        df = df.drop(columns=['kmer_usage_vec'])
    
    return df

Load the data and display it with the features

In [4]:
# features for data
df_train = extract_data_features("train.fasta")
df_train.head()

,id,seq,len,classification,gc_content,ATG_freq,GTG_freqs,TTG_freq,A_freq,G_freq,C_freq,T_freq,extra_orfs_length,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,Bacteria-4999,ATGAATCCAAGCCAAATACTTGAAAATTTAAAAAAAGAATTAAACG...,1323,0,30.234316,0.025699,0.007559,0.012094,0.424792,0.145881,0.156463,0.272865,0.301587,0.036281,0.002268,0.018896,0.011338,0.003023,0.000756,0.006803,0.005291,0.016629,0.006803,0.006047,0.016629,0.006803,0.001512,0.006047,0.003023,0.021920,0.000000,0.014361,0.003023,0.007559,0.003023,0.002268,0.001512,0.000756,0.004535,0.006047,...,0.000756,0.000000,0.000000,0.000756,0.003023,0.000756,0.000756,0.001512,0.006803,0.004535,0.000000,0.000756,0.000000,0.000756,0.000000,0.000000,0.000000,0.000000,0.000000,0.001512,0.001512,0.003779,0.000756,0.000000,0.005291,0.002268,0.000756,0.002268,0.002268,0.000756,0.000000,0.000756,0.001512,0.000000,0.000000,0.000000,0.000756,0.000000,0.000000,0.000000
1,Bacteria-5000,ATGAAGTTAAGTATCAATAAAAATACTTTAGAATCTGCAGTGATTT...,1068,0,26.310861,0.023408,0.006554,0.014045,0.421348,0.128277,0.134831,0.315543,0.308989,0.036517,0.002809,0.015918,0.009363,0.001873,0.000000,0.012172,0.004682,0.011236,0.009363,0.015918,0.008427,0.007491,0.000000,0.010300,0.001873,0.028090,0.000936,0.016854,0.003745,0.003745,0.000000,0.002809,0.000936,0.005618,0.000936,0.004682,...,0.003745,0.000000,0.000000,0.001873,0.002809,0.000936,0.000000,0.000936,0.000000,0.000936,0.000000,0.000000,0.001873,0.001873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000936,0.000000,0.000936,0.000936,0.006554,0.001873,0.000936,0.000000,0.003745,0.001873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000936,0.001873,0.000936,0.000000,0.000000
2,Bacteria-5001,ATGCAAGAAAATTACGGTGCGAGTAATATTAAAGTCCTAAAAGGCC...,2310,0,32.987013,0.024675,0.016017,0.021212,0.374892,0.199134,0.130736,0.295238,0.341558,0.025108,0.002165,0.016450,0.002597,0.006926,0.000000,0.006926,0.002165,0.009091,0.007359,0.009524,0.007792,0.004762,0.000433,0.010390,0.001732,0.030736,0.001299,0.019048,0.000433,0.009091,0.002165,0.009957,0.003896,0.005628,0.006061,0.010390,...,0.003896,0.003463,0.000433,0.001299,0.001299,0.002165,0.000433,0.000433,0.002597,0.001732,0.000866,0.000000,0.002165,0.001732,0.002597,0.000866,0.000866,0.001299,0.000000,0.000000,0.000000,0.001732,0.001299,0.000433,0.004329,0.000000,0.001299,0.001299,0.002165,0.002165,0.002165,0.000000,0.000000,0.000433,0.000000,0.000433,0.000866,0.000000,0.000000,0.000000
3,Bacteria-5002,GTGCTGGCCGAGGAGTTGACGTCCTATCGCGGCACCGACGAGCTGC...,603,0,69.651741,0.008292,0.021559,0.004975,0.144279,0.371476,0.325041,0.159204,0.592040,0.000000,0.003317,0.001658,0.001658,0.000000,0.000000,0.000000,0.009950,0.000000,0.006633,0.000000,0.008292,0.000000,0.004975,0.000000,0.009950,0.006633,0.019900,0.003317,0.021559,0.004975,0.003317,0.001658,0.013267,0.001658,0.018242,0.006633,...,0.001658,0.000000,0.014925,0.009950,0.000000,0.000000,0.006633,0.000000,0.004975,0.006633,0.016584,0.026534,0.001658,0.000000,0.009950,0.018242,0.004975,0.008292,0.004975,0.014925,0.008292,0.008292,0.024876,0.009950,0.000000,0.000000,0.001658,0.001658,0.003317,0.000000,0.006633,0.000000,0.016584,0.006633,0.004975,0.014925,0.000000,0.001658,0.008292,0.009950
4,Bacteria-5003,ATGCCGCTTACACCAGCCGATGTCCACAATGTGGCCTTCAGCAAGC...,819,0,69.841270,0.007326,0.008547,0.003663,0.199023,0.315018,0.383394,0.102564,0.161172,0.001221,0.008547,0.003663,0.009768,0.000000,0.000000,0.000000,0.006105,0.000000,0.004884,0.000000,0.006105,0.001221,0.000000,0.002442,0.018315,0.012210,0.023199,0.002442,0.019536,0.000000,0.001221,0.003663,0.010989,0.000000,0.006105,0.000000,...,0.002442,0.001221,0.008547,0.007326,0.003663,0.001221,0.007326,0.003663,0.006105,0.004884,0.018315,0.014652,0.000000,0.001221,0.002442,0.002442,0.00366

Drop data which isn't essential

In [5]:
data = df_train.drop(columns=['id', 'seq', 'len'])
data.head()

,classification,gc_content,ATG_freq,GTG_freqs,TTG_freq,A_freq,G_freq,C_freq,T_freq,extra_orfs_length,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,0,30.234316,0.025699,0.007559,0.012094,0.424792,0.145881,0.156463,0.272865,0.301587,0.036281,0.002268,0.018896,0.011338,0.003023,0.000756,0.006803,0.005291,0.016629,0.006803,0.006047,0.016629,0.006803,0.001512,0.006047,0.003023,0.021920,0.000000,0.014361,0.003023,0.007559,0.003023,0.002268,0.001512,0.000756,0.004535,0.006047,0.000756,0.008314,0.001512,...,0.000756,0.000000,0.000000,0.000756,0.003023,0.000756,0.000756,0.001512,0.006803,0.004535,0.000000,0.000756,0.000000,0.000756,0.000000,0.000000,0.000000,0.000000,0.000000,0.001512,0.001512,0.003779,0.000756,0.000000,0.005291,0.002268,0.000756,0.002268,0.002268,0.000756,0.000000,0.000756,0.001512,0.000000,0.000000,0.000000,0.000756,0.000000,0.000000,0.000000
1,0,26.310861,0.023408,0.006554,0.014045,0.421348,0.128277,0.134831,0.315543,0.308989,0.036517,0.002809,0.015918,0.009363,0.001873,0.000000,0.012172,0.004682,0.011236,0.009363,0.015918,0.008427,0.007491,0.000000,0.010300,0.001873,0.028090,0.000936,0.016854,0.003745,0.003745,0.000000,0.002809,0.000936,0.005618,0.000936,0.004682,0.000936,0.008427,0.000000,...,0.003745,0.000000,0.000000,0.001873,0.002809,0.000936,0.000000,0.000936,0.000000,0.000936,0.000000,0.000000,0.001873,0.001873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000936,0.000000,0.000936,0.000936,0.006554,0.001873,0.000936,0.000000,0.003745,0.001873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000936,0.001873,0.000936,0.000000,0.000000
2,0,32.987013,0.024675,0.016017,0.021212,0.374892,0.199134,0.130736,0.295238,0.341558,0.025108,0.002165,0.016450,0.002597,0.006926,0.000000,0.006926,0.002165,0.009091,0.007359,0.009524,0.007792,0.004762,0.000433,0.010390,0.001732,0.030736,0.001299,0.019048,0.000433,0.009091,0.002165,0.009957,0.003896,0.005628,0.006061,0.010390,0.000866,0.008658,0.001732,...,0.003896,0.003463,0.000433,0.001299,0.001299,0.002165,0.000433,0.000433,0.002597,0.001732,0.000866,0.000000,0.002165,0.001732,0.002597,0.000866,0.000866,0.001299,0.000000,0.000000,0.000000,0.001732,0.001299,0.000433,0.004329,0.000000,0.001299,0.001299,0.002165,0.002165,0.002165,0.000000,0.000000,0.000433,0.000000,0.000433,0.000866,0.000000,0.000000,0.000000
3,0,69.651741,0.008292,0.021559,0.004975,0.144279,0.371476,0.325041,0.159204,0.592040,0.000000,0.003317,0.001658,0.001658,0.000000,0.000000,0.000000,0.009950,0.000000,0.006633,0.000000,0.008292,0.000000,0.004975,0.000000,0.009950,0.006633,0.019900,0.003317,0.021559,0.004975,0.003317,0.001658,0.013267,0.001658,0.018242,0.006633,0.021559,0.001658,0.021559,...,0.001658,0.000000,0.014925,0.009950,0.000000,0.000000,0.006633,0.000000,0.004975,0.006633,0.016584,0.026534,0.001658,0.000000,0.009950,0.018242,0.004975,0.008292,0.004975,0.014925,0.008292,0.008292,0.024876,0.009950,0.000000,0.000000,0.001658,0.001658,0.003317,0.000000,0.006633,0.000000,0.016584,0.006633,0.004975,0.014925,0.000000,0.001658,0.008292,0.009950
4,0,69.841270,0.007326,0.008547,0.003663,0.199023,0.315018,0.383394,0.102564,0.161172,0.001221,0.008547,0.003663,0.009768,0.000000,0.000000,0.000000,0.006105,0.000000,0.004884,0.000000,0.006105,0.001221,0.000000,0.002442,0.018315,0.012210,0.023199,0.002442,0.019536,0.000000,0.001221,0.003663,0.010989,0.000000,0.006105,0.000000,0.006105,0.002442,0.012210,...,0.002442,0.001221,0.008547,0.007326,0.003663,0.001221,0.007326,0.003663,0.006105,0.004884,0.018315,0.014652,0.000000,0.001221,0.002442,0.002442,0.003663,0.002442,0.003663,0.015873,0.009768,0.006105,0.013431,0.029304,0.003663,0.002442,0.018315,0.007326,0.001221,0.003663,0.002442,0.002442,0.023199,0.006105,0.008547,0.024420,0.009768,0.000000,0.014652,0.00

Scale the data and split label and data

In [6]:
data_X = data.loc[:, data.columns != 'classification']
data_y = data.loc[:, data.columns == 'classification'].to_numpy()
scaler = StandardScaler()
data_X = scaler.fit_transform(data_X)

Create a Pytorch Dataset class

In [7]:
class DNAData(Dataset):
    """DNA dataset."""

    def __init__(self, data, labels):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        item = self.data[idx]
        item_label = self.labels[idx]


        return torch.Tensor(item), torch.Tensor(item_label)

Create the dataloaders and datasets with split data for train and validation

In [8]:
DNA_data = DNAData(data_X, data_y)
l_DNA = len(DNA_data)
train_set, val_set = torch.utils.data.random_split(DNA_data, [int(l_DNA*0.8), l_DNA- int(l_DNA*0.8)])

dataloaders =  {'train': DataLoader(train_set, batch_size=64, shuffle=True),
                  'val': DataLoader(val_set, batch_size=64, shuffle=True)}

Create a Pytorch Model

In [9]:
class DNA_model(torch.nn.Module):

    def __init__(self):
      super(DNA_model, self).__init__()
      self.linear0 = nn.Linear(data_X.shape[1], 128)
      self.bn0 = nn.BatchNorm1d(128)
      self.relu = nn.ReLU()
      self.linear1 = nn.Linear(128, 64)
      self.bn1 = nn.BatchNorm1d(64)
      self.linear2 = nn.Linear(64, 32)
      self.bn2 = nn.BatchNorm1d(32)
      self.linear3 = nn.Linear(32, 8)
      self.bn3 = nn.BatchNorm1d(8)
      self.linear4 = nn.Linear(8, 1)
      self.sigmoid = nn.Sigmoid()

      # nn.BatchNorm1d(int(hidden_number/2))

    def forward(self, x):
      x = self.linear0(x)
      x = self.bn0(x)
      x = self.relu(x)

      x = self.linear1(x)
      x = self.bn1(x)
      x = self.relu(x)

      x = self.linear2(x)
      x = self.bn2(x)
      x = self.relu(x)

      x = self.linear3(x)
      x = self.bn3(x)
      x = self.relu(x)

      x = self.linear4(x)
      x = self.sigmoid(x)
      return x

Create the training loop

In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import time
import os
import copy



def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    trg = []
    oup = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            count = 0
            

            # Iterate over data.
            for inp, labels in dataloaders[phase]:
                inp = inp.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inp)
                    preds = torch.round(outputs)
                    loss = criterion(outputs, labels.float())
                    count += len(inp)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inp.size(0)
                running_corrects += torch.sum(preds == labels.data)
                trg.append(labels.detach())
                oup.append(preds.detach())
            if phase == 'train':
                scheduler.step()
            



            epoch_loss = running_loss / (len(dataloaders[phase])*150)
            epoch_acc = running_corrects.double() / (len(dataloaders[phase])*150)
            it = [item for sublist in trg for item in sublist]
            tr = torch.tensor(it)
            ot = [item for sublist in oup for item in sublist]
            prd = torch.tensor(ot)

            epoch_f1 = f1_score(tr, prd, average='macro')
            epoch_precision = precision_score(tr, prd, average='macro')
            epoch_recall = recall_score(tr, prd, average='macro')
            epoch_accuracy = accuracy_score(tr, prd)

            # epoch_f1 = 0
            # epoch_precision = 0
            # epoch_recall = 0


            trg = []
            oup = []

            print('{} Loss: {:.4f} Acc: {:.4f} F1 score: {:.4f} Precision: {:.4f} Recall: {:.4f} '.format(
                phase, epoch_loss, epoch_accuracy, epoch_f1, epoch_precision, epoch_recall))

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val F1: {:4f}'.format(best_f1))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Train the model

In [11]:
model = DNA_model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
lr=0.005
optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = nn.BCELoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=50)

Epoch 0/49
----------
train Loss: 0.1877 Acc: 0.8020 F1 score: 0.8019 Precision: 0.8020 Recall: 0.8019 
val Loss: 0.1712 Acc: 0.8160 F1 score: 0.8160 Precision: 0.8164 Recall: 0.8162 

Epoch 1/49
----------
train Loss: 0.1394 Acc: 0.8560 F1 score: 0.8560 Precision: 0.8560 Recall: 0.8560 
val Loss: 0.1418 Acc: 0.8585 F1 score: 0.8585 Precision: 0.8591 Recall: 0.8588 

Epoch 2/49
----------
train Loss: 0.1237 Acc: 0.8746 F1 score: 0.8746 Precision: 0.8746 Recall: 0.8746 
val Loss: 0.1297 Acc: 0.8720 F1 score: 0.8719 Precision: 0.8739 Recall: 0.8725 

Epoch 3/49
----------
train Loss: 0.1078 Acc: 0.8928 F1 score: 0.8928 Precision: 0.8929 Recall: 0.8928 
val Loss: 0.1206 Acc: 0.8800 F1 score: 0.8800 Precision: 0.8806 Recall: 0.8803 

Epoch 4/49
----------
train Loss: 0.0998 Acc: 0.9011 F1 score: 0.9011 Precision: 0.9013 Recall: 0.9011 
val Loss: 0.1193 Acc: 0.8850 F1 score: 0.8850 Precision: 0.8850 Recall: 0.8851 

Epoch 5/49
----------
train Loss: 0.0942 Acc: 0.9061 F1 score: 0.9061 Preci

We managed to get 0.8999 F1 score